# 1 min data

In [ ]:
import pandas as pd
import os

root_dir = './data/one_min_data/'

def read_stock_data(stock_ticker):
    stock_dir = os.path.join(root_dir, stock_ticker)
    # print('stock dir ', stock_dir)
    stock_files = os.listdir(stock_dir)
    # print('stock_files ',stock_files)
    df = pd.DataFrame()
    for stock_file in stock_files:
        stock_full_path = os.path.join(stock_dir, stock_file)
        # print('stock_full_path ', stock_full_path)
        df_csv = pd.read_csv(stock_full_path, index_col=0)
        # print('df_csv shape ', df_csv.shape)
        df = pd.concat([df, df_csv])

    df['stock_id'] = stock_ticker
    df = df.sort_values(by="timestamp", ascending=True)
    df = df.reset_index(drop=True)
    # df = df.drop(columns=["index"])
    return df


In [ ]:
df_stock = read_stock_data('AAPL')
df_stock

In [ ]:
import matplotlib.pyplot as plt
# df_stock_mini = df_stock[0:1000]
# plt.plot(df_stock_mini['timestamp'], df_stock_mini['close'])
plt.plot(df_stock['close'])
plt.title('Price data')
plt.ylabel('price')
plt.xlabel('time')
plt.show();

In [ ]:
df = pd.read_csv('./data/one_min_data/AAPL/2008-03.csv', index_col=0)
df

In [ ]:
stock_ticker = 'AAPL'